In [5]:
import random
import tensorflow as tf
import tensorflow.keras.backend as K

import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score
from tensorflow.keras.utils import to_categorical

In [6]:
seed=2020
tf.keras.backend.clear_session()
np.random.seed(seed) # NumPy
random.seed(seed) # Python
tf.random.set_seed(seed) # Tensorflow

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [8]:
# input files provided in https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/
train = pd.read_csv('input/train.csv').drop('ID', axis=1)
train.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X289,X290,X291,X292,X293,X294,X295,X296,X297,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0

<h5>Category conversion</h5>

In [9]:
'''
Data description as provided in https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/data

All features are categorical
'''

'\nData description as provided in https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/data\n\nAll features are categorical\n'

In [10]:
# target
target_col = 'y'

In [11]:
# categorical features
cat_columns = train.drop(target_col, axis=1).columns.tolist()

# select only the columns that are not objects
cat_object_columns = []
for col in cat_columns:
    if train[col].dtypes == type(object):
        cat_object_columns.append(col)

In [12]:
cat_vars_ord = {}
columns = train.columns.tolist()
for col in range(train.shape[1]):
    if columns[col] in cat_columns:
        cat_vars_ord[col] = len(train[columns[col]].unique())

<h5>Retrieving the training, test and validation datasets</h5>

In [13]:
split_train_validation = int(train.shape[0]*0.8)
train[cat_object_columns] = train[cat_object_columns].fillna('0')

In [14]:
# ordinal encoding
enc = OrdinalEncoder().fit(train[cat_object_columns])
train[cat_object_columns] = enc.transform(train[cat_object_columns])

X_train_encoded = train.drop(target_col, axis=1).iloc[:split_train_validation].fillna(0)
X_val_encoded = train.drop(target_col, axis=1).iloc[split_train_validation:].fillna(0)

In [15]:
X_train_encoded.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X289,X290,X291,X292,X293,X294,X295,X296,X297,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,32.0,23.0,17.0,0.0,3.0,24.0,9.0,14.0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,32.0,21.0,19.0,4.0,3.0,28.0,11.0,14.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,20.0,24.0,34.0,2.0,3.0,27.0,9.0,23.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0

In [16]:
# separating the target columns that will be used in both datasets
y_train = train[target_col].iloc[:split_train_validation].fillna(0)
y_val = train[target_col].iloc[split_train_validation:].fillna(0)

<h5>Training the models</h5>

In [13]:
def create_model(num_attributes, return_proba=False, show_summary=False):
    model = Sequential()
    
    #input layer
    model.add(Dense(num_attributes, input_dim=num_attributes))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    # hidden layers
    model.add(Dense(num_attributes))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    
    model.add(Dense(num_attributes//2))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    
    model.add(Dense(num_attributes//4, activation='tanh'))
    
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse'), tf.keras.metrics.MeanAbsoluteError(name='mae')])
    
    # Visualize NN architecture
    print(model.summary())
    
    return model

<p>Model to be used by xMOAI</p>

In [14]:
model = create_model(X_train_encoded.shape[1])
model.fit(X_train_encoded, y_train, epochs=500, batch_size=32, validation_data=(X_val_encoded, y_val), verbose=True, use_multiprocessing=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 376)               141752    
_________________________________________________________________
batch_normalization (BatchNo (None, 376)               1504      
_________________________________________________________________
activation (Activation)      (None, 376)               0         
_________________________________________________________________
dropout (Dropout)            (None, 376)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 376)               141752    
_________________________________________________________________
batch_normalization_1 (Batch (None, 376)               1504      
_________________________________________________________________
activation_1 (Activation)    (None, 376)               0

106/106 [==============================] - 0s 3ms/step - loss: 68.0441 - rmse: 8.2489 - mae: 5.3576 - val_loss: 54.2898 - val_rmse: 7.3682 - val_mae: 5.6264
Epoch 84/500
106/106 [==============================] - 0s 3ms/step - loss: 66.4263 - rmse: 8.1502 - mae: 5.3062 - val_loss: 51.8025 - val_rmse: 7.1974 - val_mae: 5.3999
Epoch 85/500
106/106 [==============================] - 0s 3ms/step - loss: 67.7416 - rmse: 8.2305 - mae: 5.3267 - val_loss: 57.4873 - val_rmse: 7.5820 - val_mae: 5.6286
Epoch 86/500
106/106 [==============================] - 0s 3ms/step - loss: 65.2879 - rmse: 8.0801 - mae: 5.2306 - val_loss: 52.6950 - val_rmse: 7.2591 - val_mae: 5.5742
Epoch 87/500
106/106 [==============================] - 0s 3ms/step - loss: 67.6188 - rmse: 8.2231 - mae: 5.3245 - val_loss: 63.4632 - val_rmse: 7.9664 - val_mae: 6.2210
Epoch 88/500
106/106 [==============================] - 0s 3ms/step - loss: 67.6428 - rmse: 8.2245 - mae: 5.3311 - val_loss: 52.6130 - val_rmse: 7.2535 - val_mae: 

Epoch 179/500
106/106 [==============================] - 0s 3ms/step - loss: 49.6167 - rmse: 7.0439 - mae: 4.5624 - val_loss: 52.6917 - val_rmse: 7.2589 - val_mae: 4.9837
Epoch 180/500
106/106 [==============================] - 0s 3ms/step - loss: 49.9322 - rmse: 7.0663 - mae: 4.6342 - val_loss: 51.8942 - val_rmse: 7.2038 - val_mae: 5.0128
Epoch 181/500
106/106 [==============================] - 0s 3ms/step - loss: 49.8056 - rmse: 7.0573 - mae: 4.5464 - val_loss: 52.3397 - val_rmse: 7.2346 - val_mae: 5.0983
Epoch 182/500
106/106 [==============================] - 0s 3ms/step - loss: 49.5119 - rmse: 7.0365 - mae: 4.6107 - val_loss: 54.8088 - val_rmse: 7.4033 - val_mae: 5.2631
Epoch 183/500
106/106 [==============================] - 0s 3ms/step - loss: 50.3939 - rmse: 7.0989 - mae: 4.5725 - val_loss: 64.8664 - val_rmse: 8.0540 - val_mae: 6.0764
Epoch 184/500
106/106 [==============================] - 0s 3ms/step - loss: 48.0276 - rmse: 6.9302 - mae: 4.5176 - val_loss: 54.0603 - val_rmse:

Epoch 227/500
106/106 [==============================] - 0s 3ms/step - loss: 45.4909 - rmse: 6.7447 - mae: 4.3605 - val_loss: 58.3096 - val_rmse: 7.6361 - val_mae: 5.5237
Epoch 228/500
106/106 [==============================] - 0s 3ms/step - loss: 44.0806 - rmse: 6.6393 - mae: 4.3234 - val_loss: 66.9981 - val_rmse: 8.1852 - val_mae: 6.0175
Epoch 229/500
106/106 [==============================] - 0s 3ms/step - loss: 46.8987 - rmse: 6.8483 - mae: 4.5198 - val_loss: 56.1999 - val_rmse: 7.4967 - val_mae: 5.1873
Epoch 230/500
106/106 [==============================] - 0s 3ms/step - loss: 42.7893 - rmse: 6.5414 - mae: 4.2494 - val_loss: 54.1357 - val_rmse: 7.3577 - val_mae: 5.0705
Epoch 231/500
106/106 [==============================] - 0s 3ms/step - loss: 44.3129 - rmse: 6.6568 - mae: 4.3005 - val_loss: 56.7851 - val_rmse: 7.5356 - val_mae: 5.5033
Epoch 232/500
106/106 [==============================] - 0s 3ms/step - loss: 43.5328 - rmse: 6.5979 - mae: 4.2920 - val_loss: 60.9593 - val_rmse:

Epoch 275/500
106/106 [==============================] - 0s 3ms/step - loss: 40.2204 - rmse: 6.3420 - mae: 4.1557 - val_loss: 54.1096 - val_rmse: 7.3559 - val_mae: 5.1309
Epoch 276/500
106/106 [==============================] - 0s 3ms/step - loss: 42.9287 - rmse: 6.5520 - mae: 4.2655 - val_loss: 57.4812 - val_rmse: 7.5816 - val_mae: 5.3034
Epoch 277/500
106/106 [==============================] - 0s 3ms/step - loss: 39.9831 - rmse: 6.3232 - mae: 4.1211 - val_loss: 55.9032 - val_rmse: 7.4768 - val_mae: 5.2408
Epoch 278/500
106/106 [==============================] - 0s 3ms/step - loss: 39.8714 - rmse: 6.3144 - mae: 4.0854 - val_loss: 67.3632 - val_rmse: 8.2075 - val_mae: 5.9837
Epoch 279/500
106/106 [==============================] - 0s 3ms/step - loss: 39.1464 - rmse: 6.2567 - mae: 4.0908 - val_loss: 65.8643 - val_rmse: 8.1157 - val_mae: 5.9309
Epoch 280/500
106/106 [==============================] - 0s 3ms/step - loss: 38.9533 - rmse: 6.2413 - mae: 4.0794 - val_loss: 61.1803 - val_rmse:

Epoch 323/500
106/106 [==============================] - 0s 3ms/step - loss: 36.8539 - rmse: 6.0707 - mae: 3.9569 - val_loss: 54.0865 - val_rmse: 7.3544 - val_mae: 5.1245
Epoch 324/500
106/106 [==============================] - 0s 3ms/step - loss: 38.0759 - rmse: 6.1706 - mae: 4.0252 - val_loss: 55.6065 - val_rmse: 7.4570 - val_mae: 5.1955
Epoch 325/500
106/106 [==============================] - 0s 3ms/step - loss: 39.5571 - rmse: 6.2894 - mae: 4.0685 - val_loss: 58.8875 - val_rmse: 7.6738 - val_mae: 5.4025
Epoch 326/500
106/106 [==============================] - 0s 3ms/step - loss: 39.1379 - rmse: 6.2560 - mae: 4.0785 - val_loss: 52.8832 - val_rmse: 7.2721 - val_mae: 5.0145
Epoch 327/500
106/106 [==============================] - 0s 3ms/step - loss: 38.3042 - rmse: 6.1890 - mae: 4.0269 - val_loss: 56.0647 - val_rmse: 7.4876 - val_mae: 5.1046
Epoch 328/500
106/106 [==============================] - 0s 3ms/step - loss: 38.3615 - rmse: 6.1937 - mae: 4.0737 - val_loss: 57.0528 - val_rmse:

Epoch 371/500
106/106 [==============================] - 0s 3ms/step - loss: 35.7990 - rmse: 5.9832 - mae: 3.8921 - val_loss: 53.4217 - val_rmse: 7.3090 - val_mae: 5.0664
Epoch 372/500
106/106 [==============================] - 0s 3ms/step - loss: 34.4787 - rmse: 5.8719 - mae: 3.8302 - val_loss: 61.3092 - val_rmse: 7.8300 - val_mae: 5.5898
Epoch 373/500
106/106 [==============================] - 0s 3ms/step - loss: 36.5581 - rmse: 6.0463 - mae: 3.9181 - val_loss: 54.8305 - val_rmse: 7.4048 - val_mae: 5.2359
Epoch 374/500
106/106 [==============================] - 0s 3ms/step - loss: 34.9079 - rmse: 5.9083 - mae: 3.8606 - val_loss: 57.3123 - val_rmse: 7.5705 - val_mae: 5.4327
Epoch 375/500
106/106 [==============================] - 0s 3ms/step - loss: 36.4523 - rmse: 6.0376 - mae: 3.9149 - val_loss: 61.3151 - val_rmse: 7.8304 - val_mae: 5.6198
Epoch 376/500
106/106 [==============================] - 0s 3ms/step - loss: 36.3608 - rmse: 6.0300 - mae: 3.8777 - val_loss: 59.2786 - val_rmse:

Epoch 419/500
106/106 [==============================] - 0s 3ms/step - loss: 37.8984 - rmse: 6.1562 - mae: 3.9317 - val_loss: 59.6542 - val_rmse: 7.7236 - val_mae: 5.4485
Epoch 420/500
106/106 [==============================] - 0s 3ms/step - loss: 35.9831 - rmse: 5.9986 - mae: 3.8996 - val_loss: 54.9335 - val_rmse: 7.4117 - val_mae: 5.2242
Epoch 421/500
106/106 [==============================] - 0s 3ms/step - loss: 35.6405 - rmse: 5.9700 - mae: 3.9053 - val_loss: 61.3817 - val_rmse: 7.8346 - val_mae: 5.4733
Epoch 422/500
106/106 [==============================] - 0s 3ms/step - loss: 35.7539 - rmse: 5.9795 - mae: 3.8792 - val_loss: 58.6768 - val_rmse: 7.6601 - val_mae: 5.3099
Epoch 423/500
106/106 [==============================] - 0s 3ms/step - loss: 34.1934 - rmse: 5.8475 - mae: 3.7794 - val_loss: 54.6352 - val_rmse: 7.3916 - val_mae: 5.0695
Epoch 424/500
106/106 [==============================] - 0s 3ms/step - loss: 33.9390 - rmse: 5.8257 - mae: 3.7698 - val_loss: 56.7432 - val_rmse:

Epoch 467/500
106/106 [==============================] - 0s 3ms/step - loss: 33.3327 - rmse: 5.7734 - mae: 3.6573 - val_loss: 58.8260 - val_rmse: 7.6698 - val_mae: 5.3253
Epoch 468/500
106/106 [==============================] - 0s 3ms/step - loss: 33.6367 - rmse: 5.7997 - mae: 3.7373 - val_loss: 54.3700 - val_rmse: 7.3736 - val_mae: 5.0572
Epoch 469/500
106/106 [==============================] - 0s 3ms/step - loss: 33.5040 - rmse: 5.7883 - mae: 3.7102 - val_loss: 57.6305 - val_rmse: 7.5915 - val_mae: 5.2422
Epoch 470/500
106/106 [==============================] - 0s 3ms/step - loss: 34.6890 - rmse: 5.8897 - mae: 3.7695 - val_loss: 56.9775 - val_rmse: 7.5483 - val_mae: 5.2989
Epoch 471/500
106/106 [==============================] - 0s 3ms/step - loss: 34.2347 - rmse: 5.8510 - mae: 3.7357 - val_loss: 58.3252 - val_rmse: 7.6371 - val_mae: 5.4524
Epoch 472/500
106/106 [==============================] - 0s 3ms/step - loss: 33.2466 - rmse: 5.7660 - mae: 3.7543 - val_loss: 58.4054 - val_rmse:

In [15]:
model.save('model_reg')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_reg\assets


<h5>LightGBM and XGBoost</h5>

In [17]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [21]:
xgb = XGBRegressor(n_estimators=10000, n_jobs=-1, random_state=seed).fit(X_train_encoded.values, y_train, verbose=True)
mean_squared_error(y_val, xgb.predict(X_val_encoded.values), squared=False)

8.993666201760531

In [19]:
lgb = LGBMRegressor(n_estimators=10000, n_jobs=-1, random_state=seed).fit(X_train_encoded, y_train, verbose=True)
mean_squared_error(y_val, lgb.predict(X_val_encoded), squared=False)

8.744965604106822

<h5>scikit-learn</h5>

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
rf = RandomForestRegressor(n_estimators=10000, n_jobs=-1, random_state=seed).fit(X_train_encoded, y_train)
mean_squared_error(y_val, rf.predict(X_val_encoded), squared=False)

7.734661341115885

In [22]:
import pickle
pickle.dump([xgb, lgb, rf, X_val_encoded, y_val, cat_columns, cat_object_columns, cat_vars_ord], open('training_data.pkl', 'wb'))